<a href="https://colab.research.google.com/github/shaileyj/CS175_project/blob/main/stock_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/CS175/sentiment_stock_data

/content/drive/.shortcut-targets-by-id/1aoBwsHSUEGjuTzt_y5Qkgy3VlHPSbSfQ/sentiment_stock_data


In [ ]:
%ls

AAPL_sentiment.csv  COST_sentiment.csv  MSFT_sentiment.csv  TSLA_sentiment.csv
AMD_sentiment.csv   DIS_sentiment.csv   NFLX_sentiment.csv
AMZN_sentiment.csv  GOOG_sentiment.csv  PG_sentiment.csv
BA_sentiment.csv    META_sentiment.csv  PYPL_sentiment.csv


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering

In [ ]:
dfs = []
start = pd.to_datetime('2021-09-30')
end = pd.to_datetime('2022-09-30')

for filename in os.listdir():
  stock = filename.split('_')[0]
  df = pd.read_csv(filename, parse_dates=['datetime'])
  df = df[['datetime', 'open']]
  df = df[(df['datetime'] >= start) & (df['datetime'] <= end)]
  df = df.sort_values('datetime')
  df = df.rename(columns={'open': stock})
  df = df.set_index('datetime')
  dfs.append(df)

In [ ]:
aligned_prices = pd.concat(dfs, axis=1, join="inner")
returns = np.log(aligned_prices).diff().dropna()
corrs = returns.corr()
distances = np.sqrt(2 * (1 - corrs))

In [ ]:
n_clusters = 6
clustering = AgglomerativeClustering(
    metric='precomputed',
    linkage='average',
    n_clusters=n_clusters
)
labels = clustering.fit_predict(distances)

In [ ]:
clusters = pd.DataFrame({
    'Stock': corrs.index,
    'Cluster': labels
})
clusters

,Stock,Cluster
0,AAPL,1
1,AMD,1
2,AMZN,1
3,BA,0
4,COST,5
5,DIS,0
6,GOOG,1
7,META,2
8,MSFT,1
9,NFLX,3
